In [1]:
!pip install pygame

In [2]:
import time
import copy
import pygame
import sys
from statistics import median
global numar_noduri

pygame 2.1.2 (SDL 2.0.18, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [16]:


def elem_identice(lista):
    if(len(set(lista))==1) :
        return lista[0] if lista[0]!=Joc.GOL else False
    return False


class Joc:
    """
    Clasa care defineste jocul. Se va schimba de la un joc la altul.
    """
    JMIN=None
    JMAX=None
    GOL='#'
    NR_LINII=None
    NR_COLOANE=None
    scor_maxim=0
    def __init__(self, matr=None, NR_LINII=None, NR_COLOANE=None):
        #creez proprietatea ultima_mutare # (l,c)
        self.ultima_mutare=None

        if matr:  
            #e data tabla, deci suntem in timpul jocului
            self.matr=matr 
        else:
            #nu e data tabla deci suntem la initializare
            self.matr= [[self.__class__.GOL] * NR_COLOANE for i in range(NR_LINII)]
         
            if NR_LINII is not None:
                self.__class__.NR_LINII= NR_LINII
            if NR_COLOANE is not None:
                self.__class__.NR_COLOANE= NR_COLOANE
            
            ######## calculare scor maxim ###########
            sc_randuri=(NR_COLOANE-3)*NR_LINII
            sc_coloane = (NR_LINII - 3)*NR_COLOANE
            sc_diagonale = (NR_LINII - 3) * (NR_COLOANE - 3) * 2
            self.__class__.scor_maxim = sc_randuri + sc_coloane + sc_diagonale

    #conditii
    def deseneaza_grid(self, coloana_marcaj=(None,None),list_desenare =[4*(None,None)]): # tabla de exemplu este ["#","x","#","0",......]
        
        for ind in range(self.__class__.NR_COLOANE*self.__class__.NR_LINII):
            linie=ind // self.__class__.NR_COLOANE # // inseamna div
            coloana=ind % self.__class__.NR_COLOANE
            
            
            if linie == coloana_marcaj[0] and coloana == coloana_marcaj[1]:
                #daca am o patratica selectata, o desenez cu verde
                culoare=(0,255,0)
            else:
                #altfel o desenez cu alb
                culoare=(255,255,255)
            for ind2 in list_desenare:
                if linie == ind2[0] and coloana == ind2[1]:
                    culoare = (255,255,0)
            pygame.draw.rect(self.__class__.display, culoare, self.__class__.celuleGrid[ind]) #alb = (255,255,255)
            if self.matr[linie][coloana]=='x':
                self.__class__.display.blit(self.__class__.x_img,(coloana*(self.__class__.dim_celula+1),linie*(self.__class__.dim_celula+1)))
            elif self.matr[linie][coloana]=='0':
                self.__class__.display.blit(self.__class__.zero_img,(coloana*(self.__class__.dim_celula+1),linie*(self.__class__.dim_celula+1)))
        #pygame.display.flip()            
        pygame.display.update()            

    @classmethod
    def jucator_opus(cls, jucator):
        return cls.JMAX if jucator==cls.JMIN else cls.JMIN


    @classmethod
    def initializeaza(cls, display, NR_LINII=0, NR_COLOANE=0, dim_celula=100):
        cls.display=display
        cls.dim_celula=dim_celula
        cls.x_img = pygame.image.load('ics.png')
        cls.x_img = pygame.transform.scale(cls.x_img, (dim_celula,dim_celula))
        cls.zero_img = pygame.image.load('zero.png')
        cls.zero_img = pygame.transform.scale(cls.zero_img, (dim_celula,dim_celula))
        cls.celuleGrid=[] #este lista cu patratelele din grid
        for linie in range(NR_LINII):
            for coloana in range(NR_COLOANE):
                patr = pygame.Rect(coloana*(dim_celula+1), linie*(dim_celula+1), dim_celula, dim_celula)
                cls.celuleGrid.append(patr)

                
    def parcurgere(self, directie):
        um = self.ultima_mutare # (l,c)
        culoare = self.matr[um[0]][um[1]]
        nr_mutari = 0
        while True:
            um = (um[0] + directie[0], um[1] + directie[1])
            if not 0 <= um[0] < self.__class__.NR_LINII or not 0 <= um[1] < self.__class__.NR_COLOANE:
                break
            if not self.matr[um[0]][um[1]] == culoare:
                break
            nr_mutari += 1
        return nr_mutari
        
    def final(self):
        if not self.ultima_mutare: #daca e inainte de prima mutare
            return False
        directii = [[(0, 1), (0, -1)], [(1, 1), (-1, -1)], [(1, -1), (-1, 1)], [(1, 0), (-1, 0)]]
        um = self.ultima_mutare
        rez = False
        for per_dir in directii:
            len_culoare = self.parcurgere(per_dir[0]) + self.parcurgere(per_dir[1]) + 1 # +1 pt chiar ultima mutare
            if len_culoare >= 4:
                rez = self.matr[um[0]][um[1]]
        if(rez):
            return rez
        elif all(self.__class__.GOL not in x for x in self.matr):
            return 'remiza'
        else:
            return False
    
    #CONDITII pentru a muta
    def mutari_parcurgere(self, jucator):
        l_mutari=[]
        for j in range(self.__class__.NR_COLOANE):
            last_poz = None
            for i in range(self.__class__.NR_LINII):
                if i < 0 or j < 0 or i >= self.__class__.NR_LINII or j>= self.__class__.NR_COLOANE:
                    continue
                #verificam ce elem avem sub,deasuprea,stanga si dreapta pentru momentul in care punem pe tabla x/0
                #numaram vecinii si daca sunt mai multi vecini opusi nu putem pune
                if self.matr[i][j]==self.__class__.GOL:
                    opus=0
                    same=0
                    if j-1>=0 and j-1<self.__class__.NR_COLOANE:
                        if self.matr[i][j-1] == self.jucator_opus(jucator):
                            opus+=1
                        elif self.matr[i][j-1] == jucator:
                            same+=1
                    if j+1>=0 and j+1<self.__class__.NR_COLOANE:
                        if self.matr[i][j+1] == self.jucator_opus(jucator):
                            opus+=1
                        elif self.matr[i][j+1] == jucator:
                            same+=1
                    if i-1>=0 and i-1<self.__class__.NR_LINII:  
                        if self.matr[i-1][j] == self.jucator_opus(jucator):
                            opus+=1
                        elif self.matr[i-1][j] == jucator:
                            same+=1
                    if i+1>=0 and i+1<self.__class__.NR_LINII:   
                        if self.matr[i+1][j] == self.jucator_opus(jucator):
                            opus+=1
                        elif self.matr[i+1][j] == jucator:
                            same+=1
                        
                    if opus-same <= 1:
                        last_poz=(i,j)
                        matr_tabla_noua = copy.deepcopy(self.matr)
                        matr_tabla_noua[last_poz[0]][last_poz[1]] = jucator
                        jn=Joc(matr_tabla_noua)
                        jn.ultima_mutare=(last_poz[0],last_poz[1])
                        l_mutari.append(jn)
                #altfel daca vrem sa mutam, verificam toate cele 8 pozitii pe care muta
                #si le memoram pe cele in care a
                elif self.matr[i][j] == jucator:
                    for x in range(i-1,i+2):
                        for y in range(j-1,j+2):
                            if x < 0 or y < 0 or x >= self.__class__.NR_LINII or y>= self.__class__.NR_COLOANE:
                                continue
                            if self.matr[x][y]==self.__class__.GOL:
                                last_poz=(i,j)
                                matr_tabla_noua = copy.deepcopy(self.matr)
                                matr_tabla_noua[last_poz[0]][last_poz[1]] = jucator
                                jn=Joc(matr_tabla_noua)  
                                jn.ultima_mutare=(last_poz[0],last_poz[1])
                                l_mutari.append(jn)
            
            
            if last_poz is None:
                last_poz = (self.__class__.NR_LINII-1, j)
                
                
#             matr_tabla_noua = copy.deepcopy(self.matr)
#             matr_tabla_noua[last_poz[0]][last_poz[1]] = jucator
#             jn=Joc(matr_tabla_noua)
#             jn.ultima_mutare=(last_poz[0],last_poz[1])
            
#             l_mutari.append(jn)
            
        return l_mutari


    #linie deschisa inseamna linie pe care jucatorul mai poate forma o configuratie castigatoare
    #practic e o linie fara simboluri ale jucatorului opus
    def linie_deschisa(self,lista, jucator):
        jo=self.jucator_opus(jucator)
        #verific daca pe linia data nu am simbolul jucatorului opus
        if not jo in lista:
                #daca nu avem jucator opus in cele 4 casute, returnam numarul pieselor noastre( asta e ponderea )
                return lista.count(jucator)
        return 0
     #linie deschisa 2 estimare2
#     def linie_deschisa2(self,lista, jucator):
#         jo=self.jucator_opus(jucator)
#         #verific daca pe linia data nu am simbolul jucatorului opus
#         if not jo in lista:
#                 return 1
#         return 0
    def linie_deschisa_culoare(self,lista, jucator):
        jo=self.jucator_opus(jucator)
        #verific daca pe linia data nu am simbolul jucatorului opus
        if not jo in lista:
                #daca nu avem jucator opus in cele 4 casute, returnam numarul pieselor noastre( asta e ponderea )
                return lista.count(jucator)
        return 0
    def linii_deschise_culoare(self, jucator):
        list_castigatoare=[]
        linii = 0
        #luam o coloana si verificam ce pondere are in functia linie_deschisa
        for i in range(self.__class__.NR_LINII):
            for j in range(self.__class__.NR_COLOANE - 3):
                if self.linie_deschisa_culoare(self.matr[i][j:j + 4], jucator) == 4:
                    list_castigatoare = [(i,j),(i,j+1),(i,j+2),(i,j+3)]
                    return list_castigatoare
        #luam o linie si verificam ce pondere are in functia linie_deschisa        
        for j in range(self.__class__.NR_COLOANE):
            for i in range(self.__class__.NR_LINII - 3):
                if self.linie_deschisa_culoare([self.matr[k][j] for k in range(i, i + 4)],jucator):
                    list_castigatoare = [(i,j),(i+1,j),(i+2,j),(i+3,j)]
                    return list_castigatoare

        #luam diagonala secundara si verificam ce pondere are in functia linie_deschisa 
        for i in range(self.__class__.NR_LINII - 3):
            for j in range(self.__class__.NR_COLOANE - 3):
                if self.linie_deschisa_culoare([self.matr[i+k][j+k] for k in range(0, 4)],jucator):
                    list_castigatoare = [(i,j),(i+1,j+1),(i+2,j+2),(i+3,j+3)]
                    return list_castigatoare
        
        #luam diagonala principala si verificam ce pondere are in functia linie_deschisa 
        for i in range(self.__class__.NR_LINII-3):
            for j in range(3, self.__class__.NR_COLOANE):
                if self.linie_deschisa_culoare([self.matr[i+k][j-k] for k in range(0, 4)],jucator):
                    list_castigatoare = [(i,j),(i+1,j-1),(i+2,j-2),(i+3,j-3)]
                    return list_castigatoare
        
            
    def linii_deschise(self, jucator):
        
        linii = 0
        #luam o coloana si verificam ce pondere are in functia linie_deschisa
        for i in range(self.__class__.NR_LINII):
            for j in range(self.__class__.NR_COLOANE - 3):
                linii += self.linie_deschisa(self.matr[i][j:j + 4], jucator)
        #luam o linie si verificam ce pondere are in functia linie_deschisa        
        for j in range(self.__class__.NR_COLOANE):
            for i in range(self.__class__.NR_LINII - 3):
                linii += self.linie_deschisa([self.matr[k][j] for k in range(i, i + 4)],jucator)

        #luam diagonala secundara si verificam ce pondere are in functia linie_deschisa 
        for i in range(self.__class__.NR_LINII - 3):
            for j in range(self.__class__.NR_COLOANE - 3):
                linii += self.linie_deschisa([self.matr[i+k][j+k] for k in range(0, 4)],jucator)
        
        #luam diagonala principala si verificam ce pondere are in functia linie_deschisa 
        for i in range(self.__class__.NR_LINII-3):
            for j in range(3, self.__class__.NR_COLOANE):
                linii += self.linie_deschisa([self.matr[i+k][j-k] for k in range(0, 4)],jucator)
        
        return linii   
        
    def estimeaza_scor(self, adancime):
        t_final=self.final()
        #if (adancime==0):
        if t_final==self.__class__.JMAX :
            return (self.__class__.scor_maxim+adancime)
        
        elif t_final==self.__class__.JMIN:
            return (-self.__class__.scor_maxim-adancime)
        
        elif t_final=='remiza':
            return 0
        
        else:
            return (self.linii_deschise(self.__class__.JMAX)- self.linii_deschise(self.__class__.JMIN))
            
    def sirAfisare(self):
        sir="  |"
        sir+=" ".join([str(i) for i in range(self.NR_COLOANE)])+"\n"
        sir+="-"*(self.NR_COLOANE+1)*2+"\n"
        sir+= "\n".join([str(i)+" |"+" ".join([str(x) for x in self.matr[i]]) for i in range(len(self.matr))])
        return sir

    def __str__(self):
        return self.sirAfisare()

    def __repr__(self):
        return self.sirAfisare()    


In [4]:
class Stare:
    """
    Clasa folosita de algoritmii minimax si alpha-beta
    Are ca proprietate tabla de joc
    Functioneaza cu conditia ca in cadrul clasei Joc sa fie definiti JMIN si JMAX (cei doi jucatori posibili)
    De asemenea cere ca in clasa Joc sa fie definita si o metoda numita mutari() care ofera lista cu configuratiile posibile in urma mutarii unui jucator
    """
    def __init__(self, tabla_joc, j_curent, adancime, parinte=None, scor=None):
        self.tabla_joc=tabla_joc
        self.j_curent=j_curent
        
        #adancimea in arborele de stari
        self.adancime=adancime    
        
        #scorul starii (daca e finala) sau al celei mai bune stari-fiice (pentru jucatorul curent)
        self.scor=scor
        
        #lista de mutari posibile din starea curenta
        self.mutari_posibile=[]
        
        #cea mai buna mutare din lista de mutari posibile pentru jucatorul curent
        self.stare_aleasa=None



    def mutari(self):        
        l_mutari=self.tabla_joc.mutari_parcurgere(self.j_curent)
        juc_opus=Joc.jucator_opus(self.j_curent)
        l_stari_mutari=[Stare(mutare, juc_opus, self.adancime-1, parinte=self) for mutare in l_mutari]

        return l_stari_mutari
        
    
    def __str__(self):
        sir= str(self.tabla_joc) + "(Juc curent:"+self.j_curent+")\n"
        return sir    
    def __repr__(self):
        sir= str(self.tabla_joc) + "(Juc curent:"+self.j_curent+")\n"
        return sir
 

In [5]:
""" Algoritmul MinMax """

def min_max(stare):
    
    global numar_noduri
    
    if stare.adancime==0 or stare.tabla_joc.final() :
        stare.scor=stare.tabla_joc.estimeaza_scor(stare.adancime)
        return stare
    
    #calculez toate mutarile posibile din starea curenta
    stare.mutari_posibile=stare.mutari()
    numar_noduri += len(stare.mutari_posibile)
    #aplic algoritmul minimax pe toate mutarile posibile (calculand astfel subarborii lor)
    mutari_scor=[min_max(mutare) for mutare in stare.mutari_posibile]
    


    if stare.j_curent==Joc.JMAX :
        #daca jucatorul e JMAX aleg starea-fiica cu scorul maxim
        stare.stare_aleasa=max(mutari_scor, key=lambda x: x.scor)
    else:
        #daca jucatorul e JMIN aleg starea-fiica cu scorul minim
        stare.stare_aleasa=min(mutari_scor, key=lambda x: x.scor)
    stare.scor=stare.stare_aleasa.scor
    return stare

In [6]:
"""Algoritm Alpha-beta"""
def alpha_beta(alpha, beta, stare):
    global numar_noduri
    if stare.adancime==0 or stare.tabla_joc.final() :
        stare.scor=stare.tabla_joc.estimeaza_scor(stare.adancime)
        return stare
    
    if alpha>beta:
        return stare #este intr-un interval invalid deci nu o mai procesez
    
    stare.mutari_posibile=stare.mutari()
    numar_noduri += len(stare.mutari_posibile)

    if stare.j_curent==Joc.JMAX :
        scor_curent=float('-inf')
        
        for mutare in stare.mutari_posibile:
            #calculeaza scorul
            stare_noua=alpha_beta(alpha, beta, mutare)
            
            if (scor_curent<stare_noua.scor):
                stare.stare_aleasa=stare_noua
                scor_curent=stare_noua.scor
            if(alpha<stare_noua.scor):
                alpha=stare_noua.scor
                if alpha>=beta:
                    break

    elif stare.j_curent==Joc.JMIN :
        scor_curent=float('inf')
        
        for mutare in stare.mutari_posibile:
            
            stare_noua=alpha_beta(alpha, beta, mutare)
            
            if (scor_curent>stare_noua.scor):
                stare.stare_aleasa=stare_noua
                scor_curent=stare_noua.scor

            if(beta>stare_noua.scor):
                beta=stare_noua.scor
                if alpha>=beta:
                    break
    stare.scor=stare.stare_aleasa.scor

    return stare


In [7]:
def afis_daca_final(stare_curenta):
    final=stare_curenta.tabla_joc.final()
    if(final):
        if (final=="remiza"):
            print("Remiza!")
        else:
            print("A castigat "+final)
            
        return True
        
    return False

class Buton:
    def __init__(self, display=None, left=0, top=0, w=0, h=0,culoareFundal=(53,80,115), culoareFundalSel=(89,134,194), text="", font="arial", fontDimensiune=16, culoareText=(255,255,255), valoare=""):
        self.display=display        
        self.culoareFundal=culoareFundal
        self.culoareFundalSel=culoareFundalSel
        self.text=text
        self.font=font
        self.w=w
        self.h=h
        self.selectat=False
        self.fontDimensiune=fontDimensiune
        self.culoareText=culoareText
        #creez obiectul font
        fontObj = pygame.font.SysFont(self.font, self.fontDimensiune)
        self.textRandat=fontObj.render(self.text, True , self.culoareText) 
        self.dreptunghi=pygame.Rect(left, top, w, h) 
        #aici centram textul
        self.dreptunghiText=self.textRandat.get_rect(center=self.dreptunghi.center)
        self.valoare=valoare

    def selecteaza(self,sel):
        self.selectat=sel
        self.deseneaza()
        
    def selecteazaDupacoord(self,coord):
        if self.dreptunghi.collidepoint(coord):
            self.selecteaza(True)
            return True
        return False

    def updateDreptunghi(self):
        self.dreptunghi.left=self.left
        self.dreptunghi.top=self.top
        self.dreptunghiText=self.textRandat.get_rect(center=self.dreptunghi.center)

    def deseneaza(self):
        culoareF= self.culoareFundalSel if self.selectat else self.culoareFundal
        pygame.draw.rect(self.display, culoareF, self.dreptunghi)    
        self.display.blit(self.textRandat ,self.dreptunghiText) 

class GrupButoane:
    def __init__(self, listaButoane=[], indiceSelectat=0, spatiuButoane=10,left=0, top=0):
        self.listaButoane=listaButoane
        self.indiceSelectat=indiceSelectat
        self.listaButoane[self.indiceSelectat].selectat=True
        self.top=top
        self.left=left
        leftCurent=self.left
        for b in self.listaButoane:
            b.top=self.top
            b.left=leftCurent
            b.updateDreptunghi()
            leftCurent+=(spatiuButoane+b.w)

    def selecteazaDupacoord(self,coord):
        for ib,b in enumerate(self.listaButoane):
            if b.selecteazaDupacoord(coord):
                self.listaButoane[self.indiceSelectat].selecteaza(False)
                self.indiceSelectat=ib
                return True
        return False

    def deseneaza(self):
        #atentie, nu face wrap
        for b in self.listaButoane:
            b.deseneaza()

    def getValoare(self):
        return self.listaButoane[self.indiceSelectat].valoare


############# ecran initial ########################
def deseneaza_alegeri(display, tabla_curenta) :
    btn_alg=GrupButoane(
        top=30, 
        left=30,  
        listaButoane=[
            Buton(display=display, w=80, h=30, text="minimax", valoare="minimax"), 
            Buton(display=display, w=80, h=30, text="alphabeta", valoare="alphabeta")
            ],
        indiceSelectat=1)
    btn_juc=GrupButoane(
        top=100, 
        left=30, 
        listaButoane=[
            Buton(display=display, w=35, h=30, text="x", valoare="x"), 
            Buton(display=display, w=35, h=30, text="zero", valoare="0")
            ], 
        indiceSelectat=0)
    ok=Buton(display=display, top=170, left=30, w=40, h=30, text="ok", culoareFundal=(155,0,55))
    btn_alg.deseneaza()
    btn_juc.deseneaza()
    ok.deseneaza()
    while True:
        for ev in pygame.event.get(): 
            if ev.type== pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif ev.type == pygame.MOUSEBUTTONDOWN: 
                pos = pygame.mouse.get_pos()
                if not btn_alg.selecteazaDupacoord(pos):
                    if not btn_juc.selecteazaDupacoord(pos):
                        if ok.selecteazaDupacoord(pos):
                            display.fill((0,0,0)) #stergere ecran 
                            tabla_curenta.deseneaza_grid()
                            return btn_juc.getValoare(), btn_alg.getValoare()
        pygame.display.update()


In [17]:
# dc nu merge mai mult de 2
# ADANCIME_MAX=0
print(" 1 --  incepator\n","2-- mediu\n","3-- avansat")
diff=int(input("Algeti dificultatea din optiunile de mai sus: "))

if diff == 1:
    ADANCIME_MAX = 1
elif diff == 2:
    ADANCIME_MAX = 2
elif diff == 3:
    ADANCIME_MAX = 3
else:
    print("Nu ati selectat nicio varianta de mai sus!")
    sys.exit()
timp_total=time.time()
def main():
    
    
    
    
    nl=int(input("Alegeti numarul de linii a tablei de joc <= 10: ")) 
    nc=int(input("Alegeti numarul de coloane a tablei de joc >=5: "))
    #citim numarul de linii si coloane, daca nu se afla in intervalul vrut, inchidem jocul:(
    if nl >10:
        print("Nu ati introdus o valoare corespunzatoare pentru numarul de linii")
        sys.exit()
    elif nc < 5:
        print("Nu ati introdus o valoare corespunzatoare pentru numarul de coloane")
        sys.exit()
        
    #setari interf grafica
    pygame.init()
    pygame.display.set_caption("Bora Dragos-Ionut X_&_0")
    #dimensiunea ferestrei in pixeli

    w=50
    timp_calculator=[]
    len_noduri =[]
    mutari_jmax = 0
    mutari_jmin = 0
    
    ecran=pygame.display.set_mode(size=(nc*(w+1)-1,nl*(w+1)-1))# N *w+ N-1= N*(w+1)-1
    Joc.initializeaza(ecran, NR_LINII=nl, NR_COLOANE=nc, dim_celula=w)
    
    
    
    
    
    #initializare tabla
    tabla_curenta=Joc(NR_LINII=nl,NR_COLOANE=nc);
    Joc.JMIN, tip_algoritm = deseneaza_alegeri(ecran,tabla_curenta)
    print(Joc.JMIN, tip_algoritm)


    Joc.JMAX= '0' if Joc.JMIN == 'x' else 'x'
    
    

    print("Tabla initiala")
    print(str(tabla_curenta))
    
    #creare stare initiala
    stare_curenta=Stare(tabla_curenta,'x',ADANCIME_MAX)

    tabla_curenta.deseneaza_grid()
    linie_x = 0
    coloana_x = 0
    t_inainte_clk_st = None

    culoare_vecini = (255,255,0)
    v = 0
    while True :
        if v!=0:
            break
        if (stare_curenta.j_curent==Joc.JMIN):
            if t_inainte_clk_st is None:
                t_inainte_clk_st=int(round(time.time() * 1000)) 
            for event in pygame.event.get():
                
                if event.type== pygame.QUIT:
                    #iesim din program
                    pygame.quit()
                    sys.exit()
                if event.type == pygame.MOUSEMOTION:
                    
                    pos = pygame.mouse.get_pos()#coordonatele cursorului
                    for np in range(len(Joc.celuleGrid)):                        
                        if Joc.celuleGrid[np].collidepoint(pos):
                            linie=np//Joc.NR_COLOANE
                            coloana=np%Joc.NR_COLOANE
                            ###############################
                            
                            if stare_curenta.tabla_joc.matr[linie][coloana] == Joc.GOL:
                                    #verificam ce elem avem sub,deasuprea,stanga si dreapta pentru momentul in care punem pe tabla x/0
                                    #numaram vecinii si daca sunt mai multi vecini opusi nu putem pune
                                    opus=0
                                    same=0
                                    if coloana-1>=0 and coloana-1<Joc.NR_COLOANE:
                                        if stare_curenta.tabla_joc.matr[linie][coloana-1] == Joc.jucator_opus(stare_curenta.j_curent):
                                            opus+=1
                                        elif stare_curenta.tabla_joc.matr[linie][coloana-1] == Joc.JMIN:
                                            same+=1
                                    if coloana+1>=0 and coloana+1<Joc.NR_COLOANE:
                                        if stare_curenta.tabla_joc.matr[linie][coloana+1] == Joc.jucator_opus(stare_curenta.j_curent):
                                            opus+=1
                                        elif stare_curenta.tabla_joc.matr[linie][coloana+1] == Joc.JMIN:
                                            same+=1
                                    if linie-1>=0 and linie-1<Joc.NR_LINII:  
                                        if stare_curenta.tabla_joc.matr[linie-1][coloana] == Joc.jucator_opus(stare_curenta.j_curent):
                                            opus+=1
                                        elif stare_curenta.tabla_joc.matr[linie-1][coloana] == Joc.JMIN:
                                            same+=1
                                    if linie+1>=0 and linie+1<Joc.NR_LINII:   
                                        if stare_curenta.tabla_joc.matr[linie+1][coloana] == Joc.jucator_opus(stare_curenta.j_curent):
                                            opus+=1
                                        elif stare_curenta.tabla_joc.matr[linie+1][coloana] == Joc.JMIN:
                                            same+=1
                                    if opus-same <= 1:
                                        stare_curenta.tabla_joc.deseneaza_grid(coloana_marcaj=(np//Joc.NR_COLOANE,np%Joc.NR_COLOANE))
                                        break
                    
                     
                
                elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                    
                    pos = pygame.mouse.get_pos()#coordonatele cursorului la momentul clickului
                    for np in range(len(Joc.celuleGrid)):
                        
                        if Joc.celuleGrid[np].collidepoint(pos):
                            linie=np//Joc.NR_COLOANE
                            coloana=np%Joc.NR_COLOANE
                            ###############################
                            
                            if stare_curenta.tabla_joc.matr[linie][coloana] == Joc.GOL:
                                    #verificam ce elem avem sub,deasuprea,stanga si dreapta pentru momentul in care punem pe tabla x/0
                                    #numaram vecinii si daca sunt mai multi vecini opusi nu putem pune
                                    opus=0
                                    same=0
                                    if coloana-1>=0 and coloana-1<Joc.NR_COLOANE:
                                        if stare_curenta.tabla_joc.matr[linie][coloana-1] == Joc.jucator_opus(stare_curenta.j_curent):
                                            opus+=1
                                        elif stare_curenta.tabla_joc.matr[linie][coloana-1] == Joc.JMIN:
                                            same+=1
                                    if coloana+1>=0 and coloana+1<Joc.NR_COLOANE:
                                        if stare_curenta.tabla_joc.matr[linie][coloana+1] == Joc.jucator_opus(stare_curenta.j_curent):
                                            opus+=1
                                        elif stare_curenta.tabla_joc.matr[linie][coloana+1] == Joc.JMIN:
                                            same+=1
                                    if linie-1>=0 and linie-1<Joc.NR_LINII:  
                                        if stare_curenta.tabla_joc.matr[linie-1][coloana] == Joc.jucator_opus(stare_curenta.j_curent):
                                            opus+=1
                                        elif stare_curenta.tabla_joc.matr[linie-1][coloana] == Joc.JMIN:
                                            same+=1
                                    if linie+1>=0 and linie+1<Joc.NR_LINII:   
                                        if stare_curenta.tabla_joc.matr[linie+1][coloana] == Joc.jucator_opus(stare_curenta.j_curent):
                                            opus+=1
                                        elif stare_curenta.tabla_joc.matr[linie+1][coloana] == Joc.JMIN:
                                            same+=1

                                    if opus-same <= 1:
                                        stare_curenta.tabla_joc.matr[linie][coloana] = Joc.JMIN
                                        stare_curenta.tabla_joc.ultima_mutare = (linie,coloana)
                                    else:
                                        print("NU PUTETI FACE MUTAREA ASTA!!")
                                        continue
                    
                                            
                                    #afisarea starii jocului in urma mutarii utilizatorului
                                    print("\nTabla dupa mutarea jucatorului")
                                    print(str(stare_curenta))
                                    t_dupa=int(round(time.time() * 1000))
                                    print("Dumneavoastra ati gandit timp de "+str(t_dupa-t_inainte_clk_st)+" milisecunde.")
                                    
                                    t_inainte_clk_st = None
                                    
                                    stare_curenta.tabla_joc.deseneaza_grid(coloana_marcaj=(linie,coloana))
                                    
                                    mutari_jmin +=1
                                    
                                    #testez daca jocul a ajuns intr-o stare finala
                                    #si afisez un mesaj corespunzator in caz ca da
                                    if (afis_daca_final(stare_curenta)):
                                        v=1
                                        l_temp=stare_curenta.tabla_joc.linii_deschise_culoare(Joc.JMIN)
                                        print(l_temp)
                                        stare_curenta.tabla_joc.deseneaza_grid(list_desenare=l_temp)
                                        timp_total2 = time.time()
                                        print("\nTIMPUL:\n\t Total de rulare:", timp_total2 - timp_total,"s\n\tMin:", min(timp_calculator), "ms\n\tMax:", max(timp_calculator),
                                              "ms\n\tMedia:", sum(timp_calculator) / len(timp_calculator), "ms\n\tMediana:", median(timp_calculator),"ms")
                                        print("\nNUMAR NODURI:\n\tMin:", min(len_noduri), "\n\tMax:", max(len_noduri),
                                              "\n\tMedia:", sum(len_noduri) / len(len_noduri), "\n\tMediana:", median(len_noduri))
                                        break
                        

                                    #S-a realizat o mutare. Schimb jucatorul cu cel opus
                                    stare_curenta.j_curent=Joc.jucator_opus(stare_curenta.j_curent)
                   
                                
                            
                #selectam cu click dreapta ce piesa vrem sa mutam
                elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 3:
                    
                    t_inainte=int(round(time.time() * 1000))
                    pos = pygame.mouse.get_pos()#coordonatele cursorului la momentul clickului
                    
                    for np in range(len(Joc.celuleGrid)):

                        if Joc.celuleGrid[np].collidepoint(pos):
                            linie=np//Joc.NR_COLOANE
                            coloana=np%Joc.NR_COLOANE            
                            
                            #daca piesa selectata e a noastra, retinem coordonatele
                            if stare_curenta.tabla_joc.matr[linie][coloana] == Joc.JMIN:
                                linie_x=linie
                                coloana_x=coloana
                                
                    for np in range(len(Joc.celuleGrid)):
                        

                        linie_culoare=np//Joc.NR_COLOANE
                        coloana_culoare=np%Joc.NR_COLOANE 

                        if linie_x!=0 and coloana_x!=0:       
                            if abs(coloana_culoare-coloana_x) in [0,1] and abs(linie_culoare-linie_x) in [0,1]:
                                if stare_curenta.tabla_joc.matr[linie_culoare][coloana_culoare]==Joc.GOL:

                                    if coloana_culoare-1>=0 and coloana_culoare-1<Joc.NR_COLOANE:
                                        if stare_curenta.tabla_joc.matr[linie_culoare][coloana_culoare-1] == Joc.jucator_opus(stare_curenta.j_curent):
                                            opus+=1
                                        elif stare_curenta.tabla_joc.matr[linie_culoare][coloana_culoare-1] == Joc.JMIN:
                                            same+=1
                                    if coloana_culoare+1>=0 and coloana_culoare+1<Joc.NR_COLOANE:
                                        if stare_curenta.tabla_joc.matr[linie_culoare][coloana_culoare+1] == Joc.jucator_opus(stare_curenta.j_curent):
                                            opus+=1
                                        elif stare_curenta.tabla_joc.matr[linie_culoare][coloana_culoare+1] == Joc.JMIN:
                                            same+=1
                                    if linie_culoare-1>=0 and linie_culoare-1<Joc.NR_LINII:  
                                        if stare_curenta.tabla_joc.matr[linie_culoare-1][coloana_culoare] == Joc.jucator_opus(stare_curenta.j_curent):
                                            opus+=1
                                        elif stare_curenta.tabla_joc.matr[linie_culoare-1][coloana_culoare] == Joc.JMIN:
                                            same+=1
                                    if linie_culoare+1>=0 and linie_culoare+1<Joc.NR_LINII:   
                                        if stare_curenta.tabla_joc.matr[linie_culoare+1][coloana_culoare] == Joc.jucator_opus(stare_curenta.j_curent):
                                            opus+=1
                                        elif stare_curenta.tabla_joc.matr[linie_culoare+1][coloana_culoare] == Joc.JMIN:
                                            same+=1

                                    if opus-same <= 1:
                                        pygame.draw.rect(Joc.display, culoare_vecini, Joc.celuleGrid[np]) 
                                        pygame.display.update() 
                # apasam rotita (scroll-ul) pentru a selcta casuta unde o punem
                elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 2:
                     
                    pos = pygame.mouse.get_pos()#coordonatele cursorului la momentul clickului
                    for np in range(len(Joc.celuleGrid)):
                        
                        if Joc.celuleGrid[np].collidepoint(pos):
                            linie=np//Joc.NR_COLOANE
                            coloana=np%Joc.NR_COLOANE  
                            
                            if linie_x != 0 and coloana_x !=0:
                                #verificam daca pozitia selctata e in jurul piesei noastre
                                if abs(coloana-coloana_x) in [0,1] and abs(linie-linie_x) in [0,1]:
                                    if stare_curenta.tabla_joc.matr[linie][coloana]==Joc.GOL:
                                        opus=0
                                    same=0
                                    if coloana-1>=0 and coloana-1<Joc.NR_COLOANE:
                                        if stare_curenta.tabla_joc.matr[linie][coloana-1] == Joc.jucator_opus(stare_curenta.j_curent):
                                            opus+=1
                                        elif stare_curenta.tabla_joc.matr[linie][coloana-1] == Joc.JMIN:
                                            same+=1
                                    if coloana+1>=0 and coloana+1<Joc.NR_COLOANE:
                                        if stare_curenta.tabla_joc.matr[linie][coloana+1] == Joc.jucator_opus(stare_curenta.j_curent):
                                            opus+=1
                                        elif stare_curenta.tabla_joc.matr[linie][coloana+1] == Joc.JMIN:
                                            same+=1
                                    if linie-1>=0 and linie-1<Joc.NR_LINII:  
                                        if stare_curenta.tabla_joc.matr[linie-1][coloana] == Joc.jucator_opus(stare_curenta.j_curent):
                                            opus+=1
                                        elif stare_curenta.tabla_joc.matr[linie-1][coloana] == Joc.JMIN:
                                            same+=1
                                    if linie+1>=0 and linie+1<Joc.NR_LINII:   
                                        if stare_curenta.tabla_joc.matr[linie+1][coloana] == Joc.jucator_opus(stare_curenta.j_curent):
                                            opus+=1
                                        elif stare_curenta.tabla_joc.matr[linie+1][coloana] == Joc.JMIN:
                                            same+=1

                                    if opus-same <= 1:
                                        stare_curenta.tabla_joc.matr[linie][coloana] = Joc.JMIN
                                        stare_curenta.tabla_joc.matr[linie_x][coloana_x] = Joc.GOL
                                        t_dupa=int(round(time.time() * 1000))
                                        print("Dumneavoastra ati gandit timp de "+str(t_dupa-t_inainte)+" milisecunde.")
                                        stare_curenta.j_curent=Joc.jucator_opus(stare_curenta.j_curent)
                                        mutari_jmin += 1
                                        linie_x = 0
                                        coloana_x = 0
                                    else:
                                        print("NU PUTETI FACE MUTAREA ASTA!!")
                                        continue
                                        
                                else:
                                    print("NU POTI MUTA ACOLO!!")
                            else:
                                print("NU ATI SELECTAT NICIO PIESA!!")
                            if (afis_daca_final(stare_curenta)):
                                v=1
                                l_temp=stare_curenta.tabla_joc.linii_deschise_culoare(Joc.JMIN)
                                stare_curenta.tabla_joc.deseneaza_grid(list_desenare=l_temp)
                                timp_total2 = time.time()
                                print("\nTIMPUL:\n\t Total de rulare:", timp_total2 - timp_total,"s\n\tMin:", min(timp_calculator), "ms\n\tMax:", max(timp_calculator),
                                      "ms\n\tMedia:", sum(timp_calculator) / len(timp_calculator), "ms\n\tMediana:", median(timp_calculator),"ms")
                                print("\nNUMAR NODURI:\n\tMin:", min(len_noduri), "\n\tMax:", max(len_noduri),
                                      "\n\tMedia:", sum(len_noduri) / len(len_noduri), "\n\tMediana:", median(len_noduri))
                                break

    
        #--------------------------------
        else: #jucatorul e JMAX (calculatorul)
            #Mutare calculator
            
            #preiau timpul in milisecunde de dinainte de mutare
            t_inainte_calc=int(round(time.time() * 1000))
            global numar_noduri
            numar_noduri=0
            
            if tip_algoritm=='minimax':
                stare_actualizata=min_max(stare_curenta)
            else: #tip_algoritm=="alphabeta"
                stare_actualizata=alpha_beta(-500, 500, stare_curenta)
            stare_curenta.tabla_joc=stare_actualizata.stare_aleasa.tabla_joc

            print("Tabla dupa mutarea calculatorului\n"+str(stare_curenta))

            #preiau timpul in milisecunde de dupa mutare
            t_dupa=int(round(time.time() * 1000))
            print("Calculatorul a gandit timp de "+str(t_dupa-t_inainte_calc)+" milisecunde.")
            timp_calculator.append(t_dupa-t_inainte_calc)
            
            stare_curenta.tabla_joc.deseneaza_grid()
            
            len_noduri.append(numar_noduri)
            
            if (afis_daca_final(stare_curenta)):
                l_temp=stare_curenta.tabla_joc.linii_deschise_culoare(Joc.JMAX)
                print(l_temp)
                stare_curenta.tabla_joc.deseneaza_grid(list_desenare=l_temp)
                timp_total2 = time.time()
                print("\nTIMPUL:\n\t Total de rulare:", timp_total2 - timp_total,"s\n\tMin:", min(timp_calculator), "ms\n\tMax:", max(timp_calculator),
                      "ms\n\tMedia:", sum(timp_calculator) / len(timp_calculator), "ms\n\tMediana:", median(timp_calculator),"ms")
                print("\nNUMAR NODURI:\n\tMin:", min(len_noduri), "\n\tMax:", max(len_noduri),
                      "\n\tMedia:", sum(len_noduri) / len(len_noduri), "\n\tMediana:", median(len_noduri))
                break
                
            #S-a realizat o mutare. Schimb jucatorul cu cel opus
            stare_curenta.j_curent=Joc.jucator_opus(stare_curenta.j_curent)
    
if __name__ == "__main__" :
    main()
    while True :
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                
                
                pygame.quit()
                timp_total2 = time.time()
                print("\nTIMPUL:\n\t Total de rulare:", timp_total2 - timp_total,"s\n\tMin:", min(timp_calculator), "ms\n\tMax:", max(timp_calculator),
                      "ms\n\tMedia:", sum(timp_calculator) / len(timp_calculator), "ms\n\tMediana:", median(timp_calculator),"ms")
                print("\nNUMAR NODURI:\n\tMin:", min(len_noduri), "\n\tMax:", max(len_noduri),
                      "\n\tMedia:", sum(len_noduri) / len(len_noduri), "\n\tMediana:", median(len_noduri))
                sys.exit()

 1 --  incepator
 2-- mediu
 3-- avansat
Algeti dificultatea din optiunile de mai sus: 1
Alegeti numarul de linii a tablei de joc <= 10: 10
Alegeti numarul de coloane a tablei de joc >=5: 5
x minimax
Tabla initiala
  |0 1 2 3 4
------------
0 |# # # # #
1 |# # # # #
2 |# # # # #
3 |# # # # #
4 |# # # # #
5 |# # # # #
6 |# # # # #
7 |# # # # #
8 |# # # # #
9 |# # # # #

Tabla dupa mutarea jucatorului
  |0 1 2 3 4
------------
0 |# # # # #
1 |# # # # #
2 |# # # # #
3 |# x # # #
4 |# # # # #
5 |# # # # #
6 |# # # # #
7 |# # # # #
8 |# # # # #
9 |# # # # #(Juc curent:x)

Dumneavoastra ati gandit timp de 1181 milisecunde.
Tabla dupa mutarea calculatorului
  |0 1 2 3 4
------------
0 |# # # # #
1 |# # # # #
2 |# # # # #
3 |# x # # #
4 |# 0 # # #
5 |# # # # #
6 |# # # # #
7 |# # # # #
8 |# # # # #
9 |# # # # #(Juc curent:0)

Calculatorul a gandit timp de 21 milisecunde.

Tabla dupa mutarea jucatorului
  |0 1 2 3 4
------------
0 |# # # # #
1 |# # # # #
2 |# # # # #
3 |# x x # #
4 |# 0 # # #
5

NameError: name 'timp_calculator' is not defined